# Daiichi - List

## Beachten
* Für die OCR-Erkennung wurde ABBYY Fine Reader for Mac verwendet (Lizenz vorhanden)
* Wichtig: Sprachen einstellen auf: Deutsch, Englisch, Italienisch, Französisch
* Als CSV exportieren

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Unlock PDF

In [3]:
pdf = pikepdf.open('DSCH_ToV_18-06-26_001.pdf')
pdf.save('unlocked.pdf')

## OCR by Abbeyy
Sorry, you have to do this by hand: Lets run the unlocked.pdf through Abbeyy and export it to a csv. Dont forget to delete the pages at the end!

## Import CSV

In [2]:
# Read CSV into DataFrame
header = ['empty', 'name', 'address', 'plz', 'location', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'empty1', 'fees', 'related_expenses', 'empty2', 'total']  
df = pd.read_csv("DSCH_ToV_18-06-26_001.csv", sep=";", header=None, names=header)

## Format Table

In [10]:
df_export = df.copy()

#Copy empty1 if not null. But first, clean it
df_export.loc[df_export.empty1 == 'Fr.', 'empty1'] = np.NaN
df_export.loc[df_export.empty1.notna(), 'related_expenses'] = df_export.fees
df_export.loc[df_export.empty1.notna(), 'fees'] = df_export.empty1

#Remove empty
df_export.drop(columns=['empty', 'empty1', 'empty2'], inplace=True)

#Replace "nicht anwendbar"
df_export = df_export.replace("nicht anwendbar", np.NaN, regex=True)

#Remove rows which have no values
df_export = df_export.dropna(subset=['plz'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove by String Comparision
df_export = df_export[~df_export.name.str.contains('Name')]

#Add Type
df_export = set_type_by_alphabetical_order(df_export)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Clean Numbers
df_export['total'] = '0'
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)

#Fix Numbers: OCR have problems differ between "." and ",". In some cases, 100.000,00 is read like 100.000.00
#Because of that: check each number. If no ".xx" and the end, add a comma
for column in number_fields:
    df_export.loc[df_export[column] == '', column] = np.NaN
    df_export.loc[~df_export[column].str.contains(r'\.\d\d', regex=True, na=False) 
                  & df_export[column].notna(), column] = df_export[column].apply(lambda s: str(s)[:-2] + '.' + str(s)[-2:])

#Now numberize and sum
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add Country
df_export.insert(4, 'country', 'CH')
df_export.insert(4, 'uci', np.NaN)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'daiichi')

saved


In [6]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)